In [22]:
import webbrowser
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from geopy.geocoders import Nominatim
import folium

# Make pandas print all the rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


path_shape_file = "../Data/ShapeFiles/States/tl_2022_us_state.shp"

# Read the Shapefile
gdf = gpd.read_file(path_shape_file)



data = {
    'NAME': ['West Virginia', 'California', 'Texas', 'Florida'],
    'Population': [19530351, 39538223, 29145505, 21538187],
}


df_data = pd.DataFrame(data)

merged_data = gdf.merge(df_data, left_on='NAME', right_on='NAME', how='left')

print(merged_data)
# 
# # Create a map without specifying a center or zoom level
# m = folium.Map([37.090240,-95.712891], zoom_start=4)
# 
# folium.GeoJson(
#     'output.geojson',
#     style_function=lambda feature: {
#         'fillColor': 'green',  # You can customize the state colors
#         'color': 'black',
#         'weight': 1,
#         'fillOpacity': 0.6,
#         'popup': f"State: {feature['properties']['NAME']}<br>Population: {merged_data[merged_data['NAME'] == feature['properties']['NAME']]['Population'].values[0]}"
#     }
# ).add_to(m)
# 
# m

   REGION DIVISION STATEFP   STATENS GEOID STUSPS  \
0       3        5      54  01779805    54     WV   
1       3        5      12  00294478    12     FL   
2       2        3      17  01779784    17     IL   
3       2        4      27  00662849    27     MN   
4       3        5      24  01714934    24     MD   
5       1        1      44  01219835    44     RI   
6       4        8      16  01779783    16     ID   
7       1        1      33  01779794    33     NH   
8       3        5      37  01027616    37     NC   
9       1        1      50  01779802    50     VT   
10      1        1      09  01779780    09     CT   
11      3        5      10  01779781    10     DE   
12      4        8      35  00897535    35     NM   
13      4        9      06  01779778    06     CA   
14      1        2      34  01779795    34     NJ   
15      2        3      55  01779806    55     WI   
16      4        9      41  01155107    41     OR   
17      2        4      31  01779792    31    

In [ ]:
import folium
import pandas as pd
import streamlit as st
from streamlit_folium import folium_static as fs
import geopandas as gpd

def generate_popup_html(row, columns_to_display):
    # Create an empty HTML string for the popup content
    popup_html = ""

    # Iterate through the specified columns and add their values to the HTML
    for column in columns_to_display:
        popup_html += f"<b>{column}:</b> {row[column]}<br>"

    return popup_html

def load_state_map(eligibility_df: pd.DataFrame, columns_to_display):
    eligibility_df["state"] = eligibility_df["state"].astype(str).str.zfill(2)

    state_shapefile = 'Data/ShapeFiles/States/tl_2022_us_state.shp'

    # Read the Shapefile
    gdf = gpd.read_file(state_shapefile)

    merged_data = gdf.merge(eligibility_df, left_on='GEOID', right_on='state', how='left')

    # Create a map without specifying a center or zoom level
    m = folium.Map([37.090240, -95.712891], zoom_start=4)

    folium.GeoJson(
        merged_data,
        style_function=lambda feature: {
            'fillColor': 'green',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6
        },
        tooltip=folium.GeoJsonTooltip(fields=['NAME'], aliases=['State'], sticky=True),
        popup=folium.GeoJsonPopup(
            fields=columns_to_display,
            aliases=columns_to_display,  # Use the column names as aliases
            localize=True,
            labels=True,  # Display field names as labels
            parse_html=False,  # Set to True if you want to parse HTML in the columns
            max_width=500,  # Adjust max width as needed
            show=False,  # Start with the popup hidden
            max_height=None  # Optionally set a max height
        ),
    ).add_to(m)

    fs(m, width=1000, height=500)

